In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    
    tmp_inputs =
    [
        { ENTITY_NAME: "cache_data" }
    ],
    
    outputs =
    [
        { ENTITY_NAME: "bento_service" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 2_pack_bento_interface.drawio](./imgs/2_pack_bento_interface.drawio.png)

In [ ]:
#4 get substep.interface
tmp_inputs = substep.tmp_inputs()
outputs = substep.outputs()

print(f"{tmp_inputs.cache_data=}")
print(f"{outputs.bento_service=}")

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Reading basic parameters from the config

In [ ]:
import json
import os
import os.path as osp

config_fn = osp.join(tmp_inputs.cache_data, 'config.json')

with open(config_fn) as f_id:
    CONFIG = json.load(f_id)

MODEL_NAME = CONFIG["train_config_parameters"]["MODEL_NAME"]
WORK_DIR = CONFIG['work_dir'] 
# ONNX_DIR = CONFIG["onnx_dir"]
CONFIG_DEPLOY = CONFIG["config_deploy"]
DEVICE = CONFIG["device"]
CONFIG_MODEL = CONFIG["config_model"]
TORCH_MODEL = CONFIG["torch_model"]
OUT_ONNX_MODEL = CONFIG["out_onnx_model"]
TYPE_EXPORT = CONFIG["type_export"]
test_image_path = CONFIG["test_image_path"]

OUT_ONNX_MODEL

Reading config files and test image

In [ ]:
with open(config_fn, 'rb') as f_id:
    config = f_id.read()
        
with open(test_image_path, 'rb') as f_id:
    model_test_image = f_id.read()   

In [ ]:
test_image_path

#### Initializing the ONNX model via onnxruntime to calculate the result from the test image

In [ ]:
import onnxruntime
ort_session = onnxruntime.InferenceSession(OUT_ONNX_MODEL, providers=["CUDAExecutionProvider"])
input_name = ort_session.get_inputs()[0].name
output_name = [out.name for out in ort_session.get_outputs()]

In [ ]:
ort_session

### Packaging in bentoservice
with TYPE_EXPORT == "mmdeploy_export" - Model_YOLOX_Pack from ./bento_service.py is used    
with TYPE_EXPORT == "torch_export" - Model_YOLOX_NMS_Pack from ./bento_service_nms.py is used    

In [ ]:
from sinara.bentoml import save_bentoservice, load_bentoservice

In [ ]:
CONFIG_PACK = dict()
CONFIG_PACK["MAX_SIZE"] = CONFIG["train_config_parameters"]["MAX_SIZE"]
CONFIG_PACK["Normalize"] = CONFIG["train_config_parameters"]["Normalize"]
CONFIG_PACK["CLASSES"] = CONFIG["CLASSES"]
CONFIG_PACK["train_config_parameters"] = CONFIG["train_config_parameters"]
CONFIG_PACK["MIN_OBJECT_SIZE"] = CONFIG["MIN_OBJECT_SIZE"]
CONFIG_PACK["type_export"] = TYPE_EXPORT

CONFIG_PACK

In [ ]:
class_names = json.dumps({"class_names": {id_class: class_name for id_class, class_name in enumerate(CONFIG_PACK["CLASSES"])}})
class_names

In [ ]:
if TYPE_EXPORT == "mmdeploy_export":
    from bento_service import Model_YOLOX_Pack
    
    # getting the result from the test image
    from bento_service import PrePostProcessing
    
    pre_post_processing = PrePostProcessing() 
    input_data, scale_factors = pre_post_processing.prep_processing(test_image_path)
    outs = ort_session.run(output_name, {input_name: input_data})
    outs = pre_post_processing.post_processing(outs, scale_factors)
    # saving test predict result in pickle format
    import pickle
    with open(osp.join(tmp_inputs.cache_data, 'test_result.pkl'), 'wb') as pkl_file:
        pickle.dump(outs, pkl_file)
    with open(osp.join(tmp_inputs.cache_data, 'test_result.pkl'), 'rb') as f_id:
        test_result = f_id.read() 
    
    # initialize bento service
    model_service = Model_YOLOX_Pack()
    model_service.pack('model_name', MODEL_NAME)
    model_service.pack('class_names', class_names) 
    serv_v = f"{outputs.fullname('bento_service')}.{substep.run_id}"
    model_service.pack('service_version', serv_v)
    model_service.pack('model', OUT_ONNX_MODEL)
    model_service.pack('test_image', model_test_image)
    model_service.pack('config', json.dumps(CONFIG_PACK))
    model_service.pack('test_result', test_result)    
    
    # save model as a bento pack
    save_bentoservice(model_service, path=outputs.bento_service, service_version=serv_v)
    
    print("pack to bentoservice onnx model converted from mmdeploy")

In [ ]:
if TYPE_EXPORT == "torch_export":
    from bento_service_nms import Model_YOLOX_Pack
    
    # getting the result from the test image
    from bento_service_nms import PrePostProcessing
    
    pre_post_processing = PrePostProcessing(num_classes = len(CONFIG_PACK["CLASSES"])) 
    input_data, scale_factors = pre_post_processing.prep_processing(test_image_path)
    outs = ort_session.run(output_name, {input_name: input_data})
    outs = pre_post_processing.post_processing(outs, scale_factors)
    # saving test predict result in pickle format
    import pickle
    with open(osp.join(tmp_inputs.cache_data, 'test_result.pkl'), 'wb') as pkl_file:
        pickle.dump(outs, pkl_file)
    with open(osp.join(tmp_inputs.cache_data, 'test_result.pkl'), 'rb') as f_id:
        test_result = f_id.read() 
        
    # initialize bento service
    model_service = Model_YOLOX_Pack()    
    serv_v = f"{outputs.fullname('bento_service')}.{substep.run_id}"
    model_service.pack('service_version', serv_v)
    model_service.pack('model_name', MODEL_NAME)
    model_service.pack('class_names', class_names) 
    model_service.pack('model', OUT_ONNX_MODEL)
    model_service.pack('test_image', model_test_image)
    model_service.pack('config', json.dumps(CONFIG_PACK))
    model_service.pack('test_result', test_result)    
    
    # save model as a bento pack
    save_bentoservice(model_service, path=outputs.bento_service, service_version=serv_v)
    
    print("pack to bentoservice onnx model converted from torch.export")

In [ ]:
#stop spark
SinaraSpark.stop_session()